In [1]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times.

Chatbots can quickly extract important information such as demographics, symptoms,

health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access

a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

"\nChatbots are essential for speeding up user assistance and reducing waiting times.\n\nChatbots can quickly extract important information such as demographics, symptoms,\n\nhealth insurance information and assist any patients by making appointments with specialists.\n\nImagine having to design a tool that allows preliminary assistance for those who must access\n\na treatment path or must make a reservation for a specialist visit.\n\nCreate a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.\n\nThe final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.\n\nExample\nA: Hello MedAssistant.\nB: Hello. How can I help you?\nA: I don't feel well.\nB: Do you have any symptoms?\nA: I have cough and nausea.\nB: Do you want to book an appointment?\nA: Yes, please, for tomorrow.\n\n\nInfo:\n- Feel free to change or arrange a new dataset of intents\n- Try experimenting 

In [2]:
# Some helper functions

import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocessing(sentence):
    """
    params sentence: a str containing the sentence we want to preprocess
    return the tokens list
    """
    doc = nlp(sentence)
    tokens = [token.lemma_.lower()  for token in doc if not token.is_punct and not token.is_stop and not token.text.isdigit() or "not" in token.text.lower() ]
    #punct is for commas and questions marks like this kind of stuffs (punctioation)
    # we used "not" because not can block bad reactions like "not good"

    return tokens

def semantic_sim(u,v):
    return (u @ v) / (np.sqrt(sum(u**2)) * np.sqrt(sum(v**2)))

# def responce(sentence, responces):
#     s_vect = []
#     for s_token in sentence:
#         s_vect.append(s_token.vector)
#     similarities = []
#     for resp in responces:
#         r_vect = []
#         for r_token in resp:
#             r_vect.append(r_token.vector)
#         for u, v in zip(s_vect, r_vect):
#             similarities.append(semantic_sim(u, v))
        

        


In [3]:
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# STEP 0: find intents patterns

with open('med_intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        # w = preprocessing(pattern)
        w = tokenize(pattern)
        all_words.extend(w)
        patterns.append((w, tag))

print('#### WORDS')
print(all_words)
print('#### TAGS')
print(tags)
print('#### PATTERNS')
print(patterns)



#### WORDS
['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Hi', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'I', 'do', "n't", 'feel', 'good', 'I', "'m", 'not', 'doing', 'well', 'I', "'m", 'not', 'feeling', 'well', 'today', 'I', 'feel', 'sick', 'I', 'need', 'help', 'I', 'have', 'a', 'flu', 'I', 'have', 'a', 'fever', 'I', 'have', 'a', 'sore', 'throat', 'I', 'have', 'cough', 'and', 'nausia', 'I', 'have', 'a', 'migrane', 'I', 'feel', 'tired', 'I', 'have', 'a', 'cold', 'I', 'have', 'a', 'runny', 'nose', 'yes', 'yep', 'sure', 'ok', 'okay', 'please', 'sure', 'thing', 'yes', 'please', 'okay', 'please', 'no', 'nope', 'not', 'really', 'Sorry', ',', 'I', 'am', 'busy', 'I', 'am', 'busy', 'I', 'am', 'busy', 'tomorrow', 'Maybe', 'another', 'day', '?']
#### TAGS
['greeting', 'goodbye

In [4]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD

ignore_words = ['?', '!', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(patterns), "patterns")
print(len(tags), "tags")
print(len(all_words), "unique stemmed words")

45 patterns
8 tags
68 unique stemmed words


In [5]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD

num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 16
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?
# TBD
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
# for epoch in range(num_epochs):
#     for (words, labels) in train_loader:
#         words = words.to(device)
#         labels = labels.to(dtype=torch.long).to(device)

#         # Forward pass
#         outputs = model(words)
#         # if y would be one-hot, we must apply
#         # labels = torch.max(labels, 1)[1]
#         loss = criterion(outputs, labels)

#         # Backward and optimize
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

Epoch [100/1000], Loss: 0.1552
Epoch [200/1000], Loss: 0.0190
Epoch [300/1000], Loss: 0.0049
Epoch [400/1000], Loss: 0.0019
Epoch [500/1000], Loss: 0.0007
Epoch [600/1000], Loss: 0.0006
Epoch [700/1000], Loss: 0.0001
Epoch [800/1000], Loss: 0.0002
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [7]:
# STEP 5: Save the model

# TBD: name and save the model

file = "trained.pth"
torch.save(data, file)

In [8]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TBD: Load the intents file
with open('med_intents.json', 'r') as json_data:
    intents = json.load(json_data)

# TBD: Retrieve the model and all the sizings

FILE = "trained.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# TBD: build the NN

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)

bot_name = "Med_Assistant"
print("Let's have a chat! (type 'exit' to stop chatting!)")

while True:
    sentence = input("You: ")
    if sentence == "exit":
        print("Thanks for joining me!")
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print(prob)
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent['tag']:
                print(f"{bot_name}: {np.random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand. Try to be more specific")

Let's have a chat! (type 'exit' to stop chatting!)
tensor(0.9998, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: Hello, thanks for asking
tensor(0.4141, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: I do not understand. Try to be more specific
tensor(0.9999, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: What are you experiencing
tensor(0.4201, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: I do not understand. Try to be more specific
tensor(0.9997, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: Keep you warm and go to sleep
tensor(1.0000, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: Hello, thanks for asking
tensor(0.9987, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: Would you like to schedule an appointment for tomorrow?
tensor(0.9998, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: When would you like to schedule it?
tensor(0.7440, device='cuda:0', grad_fn=<SelectBackward>)
Med_Assistant: I do